In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.models.quantization as quant
from timeit import default_timer as timer

In [2]:
def print_train_time(start:float, end : float, device : torch.device = None):
    total_time = end - start
    print(f' Train time :{total_time:.3f}')
    return total_time

In [3]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = 'cpu'
model_0 = quant.resnet18(pretrained=True)

/home/tricus/task1/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/tricus/task1/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_0.parameters(), lr=0.001, momentum=0.9)

In [7]:
import torch.nn.utils.prune as prune

def prune_model(model, pruning_rate=0.3):
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):

            # Applying unstructured L1 norm pruning
            prune.l1_unstructured(module, name='weight', amount=pruning_rate)

            prune.remove(module, 'weight')

In [ ]:
model_0_training_start = timer()
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    model_0.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        model_0 = model_0.to(device)
        outputs = model_0(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}')

    if (epoch + 1) % 5 == 0:
        print(f'Pruning after epoch {epoch + 1}')
        prune_model(model_0, pruning_rate=0.1)
        print('Pruning done.')
model_0_training_end = timer()

Epoch 1, Loss: 0.5269869703487258


In [ ]:
print_train_time(start= model_0_training_start, end= model_0_training_end, device= device)

In [9]:
a = torch.arange(10)
a

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
1 - a

tensor([ 1,  0, -1, -2, -3, -4, -5, -6, -7, -8])